In [17]:
import sqlite3
from pathlib import Path
import pandas as pd

In [18]:
# Creating the file
Path('../Data/ILINet.sqlite').touch()

In [19]:
# Connecting the file
conn = sqlite3.connect('../Data/ILINet.sqlite')
c = conn.cursor()

In [20]:
# c.execute('''DROP TABLE ilinet''')

In [21]:
# Creating the table
c.execute('''CREATE TABLE ilinet (REGIONTYPE text,REGION text,YEAR int,WEEK int,
PERCENTWEIGHTEDILI float,PERCENTUNWEIGHTEDILI float,
AGE04 int,AGE2549 int,AGE2564 int,AGE524 int,AGE5064 int,AGE65 int,
ILITOTAL int,NUMOFPROVIDERS int,TOTALPATIENTS int,ID int NOT NULL PRIMARY KEY)''')

Working with the data frame to include a primary key manually

In [22]:
# load the data into a Pandas DataFrame
ili_df = pd.read_csv('../Data/ILINet.csv')
ili_df.head()

,REGIONTYPE,REGION,YEAR,WEEK,PERCENTWEIGHTEDILI,PERCENTUNWEIGHTEDILI,AGE04,AGE2549,AGE2564,AGE524,AGE5064,AGE65,ILITOTAL,NUMOFPROVIDERS,TOTALPATIENTS
0,States,Alabama,2013,40,X,1.59044,X,X,X,X,X,X,201,27,12638
1,States,Alaska,2013,40,X,0.6238,X,X,X,X,X,X,13,9,2084
2,States,Arizona,2013,40,X,0.792311,X,X,X,X,X,X,230,59,29029
3,States,Arkansas,2013,40,X,1.63326,X,X,X,X,X,X,66,18,4041
4,States,California,2013,40,X,1.52479,X,X,X,X,X,X,633,137,41514


In [23]:
# creating a list for the primary key
i = 0
id_vals=[]
for i in range(len(ili_df)):
    id_vals.append(i)
    i += 1



In [24]:
# assigning the id into the dataframe
ili_df["ID"] = id_vals

In [36]:
ili_df.head(-5)

,REGIONTYPE,REGION,YEAR,WEEK,PERCENTWEIGHTEDILI,PERCENTUNWEIGHTEDILI,AGE04,AGE2549,AGE2564,AGE524,AGE5064,AGE65,ILITOTAL,NUMOFPROVIDERS,TOTALPATIENTS,ID
0,States,Alabama,2013,40,X,1.59044,X,X,X,X,X,X,201,27,12638,0
1,States,Alaska,2013,40,X,0.6238,X,X,X,X,X,X,13,9,2084,1
2,States,Arizona,2013,40,X,0.792311,X,X,X,X,X,X,230,59,29029,2
3,States,Arkansas,2013,40,X,1.63326,X,X,X,X,X,X,66,18,4041,3
4,States,California,2013,40,X,1.52479,X,X,X,X,X,X,633,137,41514,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,States,Utah,2016,39,X,0.530623,X,X,X,X,X,X,59,39,11119,8468
8469,States,Vermont,2016,39,X,1.81564,X,X,X,X,X,X,13,4,716,8469
8470,States,Virginia,2016,39,X,1.54346,X,X,X,X,X,X,1360,136,88114,8470
8471,States,Washington,2016,39,X,0.261976,X,X,X,X,X,X,7,30,2672,8471


In [26]:
# write the data to a sqlite table
ili_df.to_sql('ilinet', conn, if_exists='append', index = False)

8478

In [27]:
# Checking that it loaded correctly
c.execute('''SELECT * FROM ilinet''').fetchone()

('States',
 'Alabama',
 2013,
 40,
 'X',
 1.59044,
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 201,
 27,
 12638,
 0)

In [28]:
# Removing most of the null data
c.execute('''ALTER TABLE ilinet DROP COLUMN REGIONTYPE''')
c.execute('''ALTER TABLE ilinet DROP COLUMN PERCENTWEIGHTEDILI''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE04''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE2549''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE2564''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE524''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE5064''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE65''')



In [29]:
# Checking for a final time
c.execute('''SELECT * FROM ilinet''').fetchone()

('Alabama', 2013, 40, 1.59044, 201, 27, 12638, 0)

Parse the db so we can load the api from it

In [30]:
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [31]:
Base = automap_base()

In [32]:
# Setting up the DB
engine = create_engine("sqlite:///../Data/ILINet.sqlite")

In [33]:
# Reflect the tables
Base.prepare(autoload_with=engine)

In [34]:
# Save the table reference
Base.classes.keys()

['ilinet']